In [ ]:
import joblib
import json
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify
from flask_cors import CORS  

# Load the trained model
try:
    model = joblib.load("churn_model.pkl")
except Exception as e:
    print(f"Error loading model: {e}")
    exit(1)

# Load the best threshold
try:
    with open("threshold.json", "r") as f:
        threshold = json.load(f)["best_threshold"]
except Exception as e:
    print(f"Error loading threshold: {e}")
    exit(1)

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable Cross-Origin Resource Sharing (Optional)

# Health Check - Root Endpoint
@app.route('/')
def home():
    return jsonify({"message": "Churn Prediction API is Running!"})

# Prediction Endpoint
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Validate JSON input
        if not request.is_json:
            return jsonify({"error": "Invalid input format. Expected JSON"}), 400
        
        data = request.get_json()

        # Expected feature order (Ensure the input matches training data format)
        feature_order = [
            'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService',
            'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
            'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
            'Contract', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges',
            'AutoPayment', 'SupportServices', 'gender_Male',
            'TenureCategory_Medium-term', 'TenureCategory_Long-term',
            'MonthlyChargesCategory_Medium', 'MonthlyChargesCategory_High',
            'MonthlyChargesCategory_Very High', 'AvgMonthlySpend',
            'LogTotalCharges', 'Monthly_Tenure', 'Support_TotalCharges'
        ]

        # Ensure all required features exist in the request
        missing_features = [feature for feature in feature_order if feature not in data]
        if missing_features:
            return jsonify({"error": f"Missing required features: {missing_features}"}), 400

        # Convert input data into a DataFrame with correct order
        input_df = pd.DataFrame([data])[feature_order]

        # Convert to NumPy array
        input_data = input_df.to_numpy()

        # Make churn probability prediction
        churn_prob = model.predict_proba(input_data)[:, 1][0]

        # Apply decision threshold
        churn_prediction = "Churn" if churn_prob >= threshold else "No Churn"

        # Return JSON response
        response = {
            "churn_probability": round(float(churn_prob), 4),  # Round to 4 decimal places
            "final_prediction": churn_prediction
        }
        return jsonify(response)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the API
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
